# Xception Model

## Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

## Data Loader

In [ ]:
class DataLoader:
  @staticmethod
  def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    return x_train, y_train, x_val, y_val, x_test, y_test

## Data Preprocessor

In [ ]:
class DataPreprocessor:
  @staticmethod
  def preprocess_data(x_train, x_val, x_test):
    x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_val = x_val.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    return x_train, x_val, x_test

## Model Builder

In [ ]:
class ModelBuilder:
  @staticmethod
  def build_model(input_shape, num_classes):
    model = models.Sequential()

    # Entry Flow
    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Middle Flow
    for _ in range(8):
      model.add(XceptionBlock(128))

    # Exit Flow
    model.add(layers.SeparableConv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.GlobalAveragePooling2D())

    # Fully connected layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

class XceptionBlock(layers.Layer):
  def __init__(self, filters):
    super(XceptionBlock, self).__init__()

    self.sep_conv1 = layers.SeparableConv2D(filters, (3, 3), activation='relu', padding='same')
    self.sep_conv2 = layers.SeparableConv2D(filters, (3, 3), activation='relu', padding='same')
    self.sep_conv3 = layers.SeparableConv2D(filters, (3, 3), activation='relu', padding='same')
    self.add_residual = layers.Add()

  def call(self, inputs):
    x = self.sep_conv1(inputs)
    x = self.sep_conv2(x)
    x = self.sep_conv3(x)
    return self.add_residual([inputs, x])

## Trainer

In [ ]:
class Trainer:
  def __init__(self, model, optimizer, loss_function, metrics):
    self.model = model
    self.model.compile(optimizer = optimizer, loss=loss_function, metrics=metrics)

  def train_model(self, x_train, y_train, x_val, y_val, epochs, batch_size):
    history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=2)
    return history

In [ ]:
def main():
  x_train, y_train, x_val, y_val, x_test, y_test = DataLoader.load_data()
  x_train, x_val, x_test = DataPreprocessor.preprocess_data(x_train, x_val, x_test)

  input_shape = x_train.shape[1:]
  num_classes = 10

  model = ModelBuilder.build_model(input_shape, num_classes)

  optimizer = 'adam'
  loss_function = 'sparse_categorical_crossentropy'
  metrics = ['accuracy']

  trainer = Trainer(model, optimizer, loss_function, metrics)

  epochs = 5
  batch_size = 32

  history = trainer.train_model(x_train, y_train, x_val, y_val, epochs, batch_size)

  test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
  print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
if __name__ == "__main__":
  main()

Epoch 1/5
1500/1500 - 104s - loss: 0.4086 - accuracy: 0.8641 - val_loss: 0.1203 - val_accuracy: 0.9638 - 104s/epoch - 69ms/step
Epoch 2/5
1500/1500 - 94s - loss: 0.1035 - accuracy: 0.9688 - val_loss: 0.1069 - val_accuracy: 0.9672 - 94s/epoch - 63ms/step
Epoch 3/5
1500/1500 - 89s - loss: 0.0707 - accuracy: 0.9782 - val_loss: 0.0636 - val_accuracy: 0.9814 - 89s/epoch - 60ms/step
Epoch 4/5
1500/1500 - 90s - loss: 0.0552 - accuracy: 0.9824 - val_loss: 0.0656 - val_accuracy: 0.9810 - 90s/epoch - 60ms/step
Epoch 5/5
1500/1500 - 90s - loss: 0.0429 - accuracy: 0.9867 - val_loss: 0.0889 - val_accuracy: 0.9724 - 90s/epoch - 60ms/step
313/313 - 7s - loss: 0.0825 - accuracy: 0.9738 - 7s/epoch - 21ms/step
Test Accuracy: 97.38%
